In [133]:
import os, sys
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd

sys.path.append('/uio/kant/geo-ceed-u1/nilscp/Nils/Python/utils')
import raster

from shapely.geometry import Point

- selection crater
- get centroid
- get old coordinate systems
- project to new coordinate system (equirectangular based on lon, lat) (centroid), I guess I have to do that for 
- apply buffer to the centroid
- feature to envelope to polygon (make a square from the buffered geometry)
- reproject to old coordinate systems
- clip based on the bounding box / geometry
- convert to ascii (but maybe not necessary with this new code)

In [62]:
# polygon layers 
path = "/uio/kant/geo-ceed-u1/nilscp/Desktop/astra/Moon/ANALYSIS/COLDSPOTS/database"
filename = os.path.join(path, 'coldspots_reproj.shp')

# centre of the crater
path_centre_crater = "/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2013_COLDSPOTS/ascii_16R"
f_centre_crater = os.path.join(path_centre_crater, 'centre_crater.csv')

# number of columns and rows
f_cols_rows = os.path.join(path_centre_crater, 'nxny.csv')

In [13]:
# reading the shape file (craters)
df = gpd.read_file(filename)

In [14]:
df.head()

,Lon,Lat,Diameter,Diameter10,CRATER_ID,MARE,BUFFER_TXT,BUFF_DIST,ORIG_FID,geometry
0,6.8320,46.9590,164,1640.0,cpcrater0000,0,164 meters,0.001475,0,"POLYGON ((206868.9726496497 1423171.562192378,..."
1,29.6445,46.7031,115,1150.0,cpcrater0001,1,115 meters,0.001034,1,"POLYGON ((898747.3938940616 1416731.563598235,..."
2,30.1152,44.0039,96,960.0,cpcrater0002,1,96 meters,0.000863,2,"POLYGON ((913633.89739661 1334816.876537439, 9..."
3,56.9245,48.1198,276,2760.0,cpcrater0003,1,276 meters,0.002482,3,"POLYGON ((1726343.590258371 1459995.239054287,..."
4,67.1107,43.1719,341,3410.0,cpcrater0004,0,341 meters,0.003067,4,"POLYGON ((2035713.944313527 1308865.995689267,..."


In [15]:
df.crs

{'proj': 'eqc',
 'lat_ts': 0,
 'lat_0': 0,
 'lon_0': 0,
 'x_0': 0,
 'y_0': 0,
 'a': 1737400,
 'b': 1737400,
 'units': 'm',
 'no_defs': True}

In [21]:
df.shape

(2282, 10)

In [77]:
# reading centre of the crater
ccrater = pd.read_csv(f_centre_crater, skiprows = 1, names=['x', 'y', 'flag'])

ccrater.head(5)

,x,y,flag
0,1206,1648,1
1,1216,1594,0
2,0,0,0
3,2371,1968,1
4,2603,3054,1


In [63]:
f_cols_rows

'/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2013_COLDSPOTS/ascii_16R/nxny.csv'

In [144]:
# reading ncols nrows
cols_rows = pd.read_csv(f_cols_rows, names=['width', 'height'])

cols_rows.head(10)

,width,height
0,355.0,355.0
1,249.0,249.0
2,209.0,208.0
3,597.0,597.0
4,738.0,738.0
5,887.0,887.0
6,1077.0,1077.0
7,824.0,825.0
8,385.0,386.0
9,781.0,781.0


In [146]:
reproj_raster

'/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2013_LARGE_CRATERS/ascii_olivine/crater0049.tif'

In [151]:
# Olivine

path_raster = '/uio/kant/geo-ceed-u1/nilscp/Desktop/lunaspec/SLDEM2015'
in_raster = os.path.join(path_raster, 'Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif')
cliptype = 'geojson'

path_mineralogy = '/uio/kant/geo-ceed-u1/nilscp/Desktop/astra/Moon/Kaguya_mineralogy/'
in_raster_miner = os.path.join(path_mineralogy, 'Lunar_Kaguya_MIMap_MineralDeconv_ClinopyroxenePercent_50N50S_clip.tif')
paths = '/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2013_COLDSPOTS/ascii_cpx'

with rio.open(in_raster) as src:
    crs_wkt_src = src.crs.to_wkt()
    
with rio.open(in_raster_miner) as src:
    crs_wkt_src_miner = src.crs.to_wkt()

for i in range(np.shape(ccrater)[0]):
#for i in range(50):    
    #print(i)
    
    if ((df.loc[i,'Lat'] >= 50.0) or (df.loc[i,'Lat'] <= -50.0)):
        None
        
    else:    
        if ccrater.loc[i,'y'] != 0:
            
            
            # get centre of the crater
            xmiddle = (cols_rows.loc[i, 'width'] * resolution_kaguya) / 2.0
            ymiddle = (cols_rows.loc[i, 'height'] * resolution_kaguya) / 2.0
            
            # check offset compare to real centre of the crater
            offsetx = ccrater.loc[i,'x']- xmiddle
            offsety = ccrater.loc[i,'y']- ymiddle

            dict_crater = {'geometry': [df.geometry.loc[i]], 'index': [0]}
            dataframe_crater = pd.DataFrame(dict_crater)
            geodataframe_crater = gpd.GeoDataFrame(dataframe_crater, geometry=dataframe_crater.geometry)
            geodataframe_crater.crs = df.crs

            geom_geoseries = gpd.GeoSeries(geodataframe_crater.geometry)
            geom_geoseries.crs = df.crs

            centroids = geom_geoseries.centroid
            new_crs = df.crs.copy()
            new_crs['lat_ts'] = int(df.Lat.loc[i])

            # get new coordinate system 
            crs_wkt_dst = raster.crs_eqc(crs_wkt_src, df.Lat.loc[i])

            # Change coordinates of centroids (need to modify the geometry here)
            # add X and Y (with the cellsize of Kaguya DEM, constant here and equal to 7.4031617
            # need to get point from shapely
            centroids_eq = centroids.to_crs(new_crs)
            
            # update centre crater value 
            centroids_eq.geometry[0] = Point(centroids_eq.geometry[0].x + offsetx, centroids_eq.geometry[0].y + offsety)

            # Does it make the buffer from the polygon? I do specifically 
            buffer_array = centroids_eq.buffer((df.Diameter.loc[i]*0.5)*1.5) # I will go for 0.75
            envelope_array = buffer_array.envelope
            envelope_global = envelope_array.to_crs(df.crs) #df.crs

            tmp = envelope_global.__geo_interface__ 
            in_polygon = [tmp['features'][0]['geometry']]

            # clip
            fname_raster_clip = 'crater' + str(i).zfill(4) + '_clip.tif'
            clipped_raster = os.path.join(paths, fname_raster_clip)
            raster.clip_advanced(in_raster_miner, in_polygon, cliptype, clipped_raster)

            # reproject
            fname_raster_reproj = 'crater' + str(i).zfill(4) + '.tif'
            reproj_raster = os.path.join(paths, fname_raster_reproj)
            raster.reproject(clipped_raster, crs_wkt_dst, reproj_raster)

        else:
            None
    
    # something to remember: the reprojection cause the raster to not have the resolution
    # there are two solutions: 1) work with rasters of different resolution or 2) downscale the raster
    # but you will miss some of the data...
    # doing 1) would be mean to change quite lot of the scripts previously written
    # but I don't think this is a problem for the mineralogical map.
    # the only question if I would have better detailed results with "native resolution"
    #raster.reproject(clipped_raster, crs_wkt_dst, reproj_raster)